In [1]:

def fft_spectrum(nd_image, padding):
    """
    Returns the FFT spectrum of an arbitrary nD image as the absolute logarithm (log1p) of the Fourier transform magnitude. Ensures that the DC component is at the center of the image. Uses reflection padding (provided as integer parameter) and Hamming apodisation to reduce artefacts due to discontinuities at the image borders.
    """
    import numpy as np

    # computing padding tuple:
    pad_width = ((padding, padding),)*nd_image.ndim

    # Padding:
    image_padded = np.pad(nd_image, pad_width=pad_width, mode='symmetric')
    
    # prepare the apodization window in 1D:
    from scipy import signal
    window_tuple = (signal.windows.hamming(length+2*padding) for length in nd_image.shape)
    
    # prepare the apodization window in nD:
    import functools
    window = functools.reduce(np.multiply.outer, window_tuple)
    
    # Apodize:
    image_padded_apodized = window*image_padded

    # Compute the Fourier transform of the image:
    image_fft = np.fft.fftn(image_padded_apodized)
    
    # Shift the zero-frequency component to the center of the spectrum:
    image_fft_shifted = np.fft.fftshift(image_fft)
    
    # Compute the magnitude spectrum:
    magnitude_spectrum = np.log1p(np.abs(image_fft_shifted))
        
    return(magnitude_spectrum)

In [2]:
def check(candidate):
    
    def fft_spectrum_reference(nd_image, padding):
        """
        Returns the FFT spectrum of an arbitrary nD image as the absolute logarithm (log1p) of the Fourier transform magnitude. Ensures that the DC component is at the center of the image. Uses reflection padding (provided as integer parameter) and Hamming apodization to reduce artefacts due to discontinuities at the image borders.
        """
        import numpy as np
    
        # computing padding tuple:
        pad_width = ((padding, padding),)*nd_image.ndim
    
        # Padding:
        image_padded = np.pad(nd_image, pad_width=pad_width, mode='symmetric')
        
        # prepare the apodization window in 1D:
        from scipy import signal
        window_tuple = (signal.windows.hamming(length+2*padding) for length in nd_image.shape)
        
        # prepare the apodization window in nD:
        import functools
        window = functools.reduce(np.multiply.outer, window_tuple)
        
        # Apodize:
        image_padded_apodized = window*image_padded
    
        # Compute the Fourier transform of the image:
        image_fft = np.fft.fftn(image_padded_apodized)
        
        # Shift the zero-frequency component to the center of the spectrum:
        image_fft_shifted = np.fft.fftshift(image_fft)
        
        # Compute the magnitude spectrum:
        magnitude_spectrum = np.log1p(np.abs(image_fft_shifted))
            
        return(magnitude_spectrum)
    
    import numpy as np   

    # Test image:
    image = np.ones((5, 6, 7), dtype=np.int16)

    # Padding:
    padding_width = 10

    # running the candidate function:
    magnitude_spectrum_candidate = candidate(image, padding_width)
    
    # Reference implementation:
    magnitude_spectrum_reference = fft_spectrum_reference(image, padding_width)
    
    assert np.allclose(magnitude_spectrum_candidate, magnitude_spectrum_reference)

In [3]:
check(fft_spectrum)

In [4]:
# # display the magnitude spectrum of a 3D image:
# import numpy as np
# import matplotlib.pyplot as plt
# 
# # Camera Test image:
# image = np.ones((50, 50, 50), dtype=np.int16)
# 
# # Fill the image with integer noise:
# image += np.random.randint(0, 10, image.shape)
# 
# # Blurr the image:
# from scipy.ndimage import gaussian_filter
# image = gaussian_filter(image, sigma=12)
# 
# # Padding:
# padding_width = 25
# 
# # Compute the magnitude spectrum:
# magnitude_spectrum = fft_spectrum(image, padding_width)
# 
# # Display the magnitude spectrum:
# #plt.imshow(magnitude_spectrum[15, :, :])
# #plt.show()
# 
# # Display the magnitude spectrum in napari
# import napari
# viewer = napari.Viewer()
# viewer.add_image(magnitude_spectrum, name='Magnitude Spectrum')
# viewer.add_image(image, name='Original Image')
# viewer.show
    
    